In [ ]:
import os
from ultralytics import YOLO
from glob import glob
from sklearn.model_selection import train_test_split
import yaml
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [ ]:
# !unzip /gemini/data-1/Fatigue.zip

In [ ]:
dataset_path = 'dataset/'
# 获取所有图像文件和标注文件路径
image_files = glob(os.path.join(dataset_path, 'images', '*.jpg'))
annotation_files = glob(os.path.join(dataset_path, 'Annotations', '*.xml'))
print(len(image_files))

In [ ]:

# # 处理没有标签的图片，只要第一次运行
# for pic in image_files:
#     anno_name = (pic.split('.')[0] + '.xml').replace('images','Annotations')
#     if anno_name not in annotation_files:
#         os.remove(os.path.join(pic))

# 手动将JPEGImages重命名为images

In [ ]:
# 转换VOC格式数据为YOLO格式
import xml.etree.ElementTree as ET

classes = ['closed_eye', 'open_eye','closed_mouth','open_mouth']  # 根据你的数据集类别修改

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(image_id):
    in_file = open(dataset_path+'Annotations/%s.xml' % (image_id))
    out_file = open(dataset_path+'labels/%s.txt' % (image_id), 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult) == 1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text),
             float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        out_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

if not os.path.exists(dataset_path+'labels/'):
    os.makedirs(dataset_path+'labels/')
for filename in annotation_files:
    if filename.endswith('.xml'):
        # file_path = os.path.join(path, filename)
        image_id = os.path.splitext(os.path.basename(filename))[0]
        convert_annotation(image_id)



In [ ]:
# 进行8:2分割

labels_files = glob(os.path.join(dataset_path, 'labels', '*.txt'))
train_images, val_images, train_labels, val_labels = train_test_split(
    image_files, labels_files, test_size=0.2, random_state=42)

In [16]:
# 将文件路径写入文本文件
dataset_path = 'datasets/'

if not os.path.exists(dataset_path+'paths/'):
    os.makedirs(dataset_path+'paths/')
def write_file_list(file_list, filename):
    filename = os.path.join(dataset_path,'paths', filename)
    with open(filename, 'w') as f:
        for file in file_list:
            f.write(f"{file}\n")

write_file_list(train_images, 'train_images.txt')
write_file_list(val_images, 'val_images.txt')
write_file_list(train_labels, 'train_labels.txt')
write_file_list(val_labels, 'val_labels.txt')

In [17]:
# 生成my_det_data.yaml文件
data = {
    'train': 'paths/train_images.txt',
    'val': 'paths/val_images.txt',
    'nc': 4,  
    'names': ['closed_eye', 'open_eye','closed_mouth','open_mouth']
}

with open('Fatigue_det_data.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)


In [18]:
# Load a model
model = YOLO("yolov8n.yaml") 

In [19]:
# Train the model
results = model.train(data="Fatigue_det_data.yaml", 
                      epochs=10, 
                      imgsz=320, 
                      batch=2,
                      workers=1)

Ultralytics YOLOv8.2.16 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (B1.gpu.small, 5922MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=Fatigue_det_data.yaml, epochs=10, time=None, patience=100, batch=2, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning dataset/labels... 2331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2331/2331 [00:04<00:00, 541.40it/s]


train: New cache created: dataset/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/root/miniconda3/lib/python3.11/site-packages/albumentations/core/composition.py:151: UserWarning: Got processor for bboxes, but no transform to process it.
  warnings.warn(f"Got processor for {proc.default_data_name}, but no transform to process it.")
val: Scanning dataset/labels... 583 images, 0 backgrounds, 0 corrupt: 100%|██████████| 583/583 [00:01<00:00, 480.46it/s]


val: New cache created: dataset/labels.cache
Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 


...

                   all        583       1572      0.829      0.825      0.906      0.436

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10     0.533G      1.836      1.213     0.9315          3        320: 100%|██████████| 1166/1166 [01:13<00:00, 15.77it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [00:04<00:00, 31.45it/s]


                   all        583       1572      0.819       0.85      0.918      0.446

10 epochs completed in 0.207 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.2MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (B1.gpu.small, 5922MiB)
YOLOv8n summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 146/146 [00:03<00:00, 38.73it/s]


                   all        583       1572      0.819       0.85      0.918      0.445
            closed_eye        583        409      0.896       0.72       0.88      0.388
              open_eye        583        595      0.815      0.801      0.875      0.391
          closed_mouth        583        516      0.925      0.975      0.985      0.529
            open_mouth        583         52      0.641      0.904      0.932      0.473
Speed: 0.1ms preprocess, 1.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train4


In [21]:
# 优化参数方案一：增加数据增强、轮数、batch
results = model.train(data="Fatigue_det_data.yaml", 
                      epochs=20, 
                      imgsz=320, 
                      batch=4,
                      workers=1,
                      mosaic=0.5,  # 增加mosaic数据增强
                      mixup=0.2,   # 增加mixup数据增强
                      copy_paste=0.2)  # 增加copy-paste数据增强


Ultralytics YOLOv8.2.16 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (B1.gpu.small, 5922MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=Fatigue_det_data.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=1, project=None, name=train42, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_

train: Scanning dataset/labels... 2331 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2331/2331 [00:03<00:00, 584.39it/s]


train: New cache created: dataset/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/root/miniconda3/lib/python3.11/site-packages/albumentations/core/composition.py:151: UserWarning: Got processor for bboxes, but no transform to process it.
  warnings.warn(f"Got processor for {proc.default_data_name}, but no transform to process it.")
val: Scanning dataset/labels... 583 images, 0 backgrounds, 0 corrupt: 100%|██████████| 583/583 [00:00<00:00, 714.89it/s]


val: New cache created: dataset/labels.cache
Plotting labels to runs/detect/train42/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 320 train, 320 val
Using 1 dataloader workers
Logging results to runs/detect/train42
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20     0.556G      1.605     0.8591     0.8954          6        320: 100%|██████████| 583/583 [00:41<00:00, 14.12it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:03<00:00, 18.57it/s]

                   all        583       1572      0.916      0.941      0.973      0.534



20 epochs completed in 0.280 hours.
Optimizer stripped from runs/detect/train42/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train42/weights/best.pt, 6.2MB

Validating runs/detect/train42/weights/best.pt...
Ultralytics YOLOv8.2.16 🚀 Python-3.11.8 torch-2.1.2+cu121 CUDA:0 (B1.gpu.small, 5922MiB)
YOLOv8n summary (fused): 168 layers, 3006428 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 73/73 [00:03<00:00, 22.30it/s]


                   all        583       1572      0.926      0.933      0.975      0.535
            closed_eye        583        409      0.909        0.9      0.962      0.486
              open_eye        583        595      0.933       0.86      0.958      0.456
          closed_mouth        583        516      0.964      0.992      0.994      0.562
            open_mouth        583         52      0.897      0.981      0.986      0.636
Speed: 0.0ms preprocess, 1.2ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to runs/detect/train42


In [11]:
! zip -r /gemini/code/runs/detect/train42.zip /gemini/code/runs/detect/train42

  adding: gemini/code/runs/detect/train42/ (stored 0%)
  adding: gemini/code/runs/detect/train42/F1_curve.png (deflated 8%)
  adding: gemini/code/runs/detect/train42/PR_curve.png (deflated 16%)
  adding: gemini/code/runs/detect/train42/P_curve.png (deflated 12%)
  adding: gemini/code/runs/detect/train42/R_curve.png (deflated 9%)
  adding: gemini/code/runs/detect/train42/args.yaml (deflated 53%)
  adding: gemini/code/runs/detect/train42/confusion_matrix.png (deflated 28%)
  adding: gemini/code/runs/detect/train42/confusion_matrix_normalized.png (deflated 25%)
  adding: gemini/code/runs/detect/train42/events.out.tfevents.1721707730.gjob-dev-470774588713136128-taskrole1-0.1387.1 (deflated 92%)
  adding: gemini/code/runs/detect/train42/labels.jpg (deflated 32%)
  adding: gemini/code/runs/detect/train42/labels_correlogram.jpg (deflated 33%)
  adding: gemini/code/runs/detect/train42/results.csv (deflated 83%)
  adding: gemini/code/runs/detect/train42/results.png (deflated 8%)
  adding: gemin

In [ ]:
! zip -r /gemini/code/runs/detect/train42.zip /gemini/code/runs/detect/train42